In [1]:
from sosmetrics.metrics import BinaryCenterMetric, PD_FAMetric
import os
from tqdm import tqdm
import cv2
import torch
import numpy as np

In [2]:
gt_path = '../testdata/labels/'
preds_path = '../testdata/preds/'

# get all imgs'name, remove the extension
images_folder1 = [f for f in os.listdir(gt_path) if
                  os.path.isfile(os.path.join(gt_path, f)) and f.lower().endswith(
                      ('.png', '.jpg', '.jpeg', '.bmp', '.tiff', '.gif'))]
images_names = [os.path.splitext(f)[0] for f in images_folder1]
tbar = tqdm(images_names)

  0%|          | 0/21 [00:00<?, ?it/s]

# Binary Center Metric

In [3]:

# For test multiple format of input.
gt_img_list = []
pred_img_list = [] 

gt_img_list_chw = []
pred_img_list_chw = []

gt_img_paths = []
pred_img_paths = []

Metric = BinaryCenterMetric(
    dis_thr=[1, 10],
    conf_thr=0.5,
    dilate_kernel_size=[7,7],
    match_alg='hungarian',
    # debug = True
    )
for image_name in tbar:
    tbar.set_description(f"Reading image_name={image_name}")
    gt_image_path = os.path.join(gt_path, f"{image_name}.png")  # 假设所有图片都是PNG格式，或者你可以根据实际情况修改
    pred_image_path = os.path.join(preds_path, f"{image_name}.png") 

    gt_img = cv2.imread(gt_image_path)
    pred_img = cv2.imread(pred_image_path)

    # for test [chw, chw, ...] format
    gt_img_list_chw.append(gt_img.transpose(2,1,0))
    pred_img_list_chw.append(pred_img.transpose(2,1,0))

    # for test [hwc, hwc] format
    gt_img_list.append(gt_img)
    pred_img_list.append(pred_img)

    # for test [path, path] format

    gt_img_paths.append(gt_image_path)
    pred_img_paths.append(pred_image_path)

    # for test single img or img path
    Metric.update(gt_img, pred_img)
    Metric.update(gt_image_path, pred_image_path)

print("Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.")
Metric.get()
Metric.reset()


print("Test image of list [hwc, hwc, ...]")
Metric.update(gt_img_list, pred_img_list) 
Metric.get()
Metric.reset()

print("Test image_path of list, [img_path, img_path, ...]")
Metric.update(gt_img_paths, pred_img_paths)
Metric.get()
Metric.reset()

print("Test image of np.array, bhwc")
Metric.update(np.stack(gt_img_list), np.stack(pred_img_list)) 
Metric.get()
Metric.reset()


print("Test image of tensor, bchw")
Metric.update(torch.from_numpy(np.stack(gt_img_list_chw)), torch.from_numpy(np.stack(pred_img_list_chw))) 
Metric.get()
Metric.reset()

print("Test image of list,  [chw, chw, ...]")
Metric.update(gt_img_list_chw, pred_img_list_chw) 
Metric.get()
Metric.reset()

Reading image_name=0020: 100%|██████████| 21/21 [00:00<00:00, 59.87it/s]


Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.
+-----------+----+----+----+------------------+---------------+----------------+
| Threshold | TP | FP | FN | target_Precision | target_Recall | target_F1score |
+-----------+----+----+----+------------------+---------------+----------------+
|     1     | 24 | 20 | 28 |     0.54545      |    0.46154    |    0.50000     |
|     2     | 38 | 6  | 14 |     0.86364      |    0.73077    |    0.79167     |
|     3     | 40 | 4  | 12 |     0.90909      |    0.76923    |    0.83333     |
|     4     | 40 | 4  | 12 |     0.90909      |    0.76923    |    0.83333     |
|     5     | 40 | 4  | 12 |     0.90909      |    0.76923    |    0.83333     |
|     6     | 42 | 2  | 10 |     0.95455      |    0.80769    |    0.87500     |
|     7     | 42 | 2  | 10 |     0.95455      |    0.80769    |    0.87500     |
|     8     | 42 | 2  | 10 |     0.95455      |    0.80769    |    0.8750

# PD_FA Metric

In [4]:
# For test multiple format of input.
from sosmetrics.metrics import PD_FAMetric


gt_img_list = []
pred_img_list = [] 

gt_img_list_chw = []
pred_img_list_chw = []

gt_img_paths = []
pred_img_paths = []

Metric = PD_FAMetric(
    dis_thr=[1, 10],
    conf_thr=0.5,
    # debug = True
    )
for image_name in tbar:
    tbar.set_description(f"Reading image_name={image_name}")
    gt_image_path = os.path.join(gt_path, f"{image_name}.png") 
    pred_image_path = os.path.join(preds_path, f"{image_name}.png") 

    gt_img = cv2.imread(gt_image_path)
    pred_img = cv2.imread(pred_image_path)

    # for test [chw, chw, ...] format
    gt_img_list_chw.append(gt_img.transpose(2,1,0))
    pred_img_list_chw.append(pred_img.transpose(2,1,0))

    # for test [hwc, hwc] format
    gt_img_list.append(gt_img)
    pred_img_list.append(pred_img)

    # for test [path, path] format
    gt_img_paths.append(gt_image_path)
    pred_img_paths.append(pred_image_path)

    # for test single img or img path
    Metric.update(gt_img, pred_img)
    Metric.update(gt_image_path, pred_image_path)

print("Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.")
Metric.get()
Metric.reset()


print("Test image of list [hwc, hwc, ...]")
Metric.update(gt_img_list, pred_img_list) 
Metric.get()
Metric.reset()

print("Test image_path of list, [img_path, img_path, ...]")
Metric.update(gt_img_paths, pred_img_paths)
Metric.get()
Metric.reset()

print("Test image of np.array, bhwc")
Metric.update(np.stack(gt_img_list), np.stack(pred_img_list)) 
Metric.get()
Metric.reset()


print("Test image of tensor, bchw")
Metric.update(torch.from_numpy(np.stack(gt_img_list_chw)), torch.from_numpy(np.stack(pred_img_list_chw))) 
Metric.get()
Metric.reset()

print("Test image of list,  [chw, chw, ...]")
Metric.update(gt_img_list_chw, pred_img_list_chw) 
Metric.get()
Metric.reset()

Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.
+-----------+----+----+------+---------+-----------+-------------+
| Threshold | TD | AT |  FD  |    NP   | target_Pd |   pixel_Fa  |
+-----------+----+----+------+---------+-----------+-------------+
|     1     | 22 | 52 | 2738 | 2752512 |  0.42308  | 9.94728e-04 |
|     2     | 40 | 52 | 668  | 2752512 |  0.76923  | 2.42687e-04 |
|     3     | 40 | 52 | 668  | 2752512 |  0.76923  | 2.42687e-04 |
|     4     | 40 | 52 | 668  | 2752512 |  0.76923  | 2.42687e-04 |
|     5     | 40 | 52 | 668  | 2752512 |  0.76923  | 2.42687e-04 |
|     6     | 42 | 52 | 368  | 2752512 |  0.80769  | 1.33696e-04 |
|     7     | 42 | 52 | 368  | 2752512 |  0.80769  | 1.33696e-04 |
|     8     | 44 | 52 |  0   | 2752512 |  0.84615  | 0.00000e+00 |
|     9     | 44 | 52 |  0   | 2752512 |  0.84615  | 0.00000e+00 |
|     10    | 44 | 52 |  0   | 2752512 |  0.84615  | 0.00000e+00 |
+-----------